<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Stance_Veracity_71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import fastai

In [0]:
from fastai.text import * 

In [0]:
def read_stance(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 3.0:
            return 'Query'
        elif col['label'] == 2.0:
            return 'Deny'
        elif col['label'] == 1.0:
            return 'Support'
        elif col['label'] == 0.0:
            return 'Comment'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [0]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [0]:
# For veracity
train = read_veracity('train.csv', True)
test = read_veracity('test.csv')

In [0]:
# For stance
train = read_stance('train_stance.csv', True)
test = read_stance('test_stance.csv')

In [0]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [6]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [7]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.631519,4.729428,0.228214,00:02


In [11]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.458423,4.066679,0.294152,00:02


In [12]:
learn.predict("This is a review about", n_words=10)

'This is a review about barney gross gross explain : actually being into a non'

In [0]:
learn.save_encoder('ft_enc')

In [14]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (3389 items)
x: TextList
xxbos @cameron_gray xxunk @globeandmail xxmaj xxunk - up for so many xxunk .,xxbos @cnni xxmaj xxunk has made xxunk # xxup xxunk # weapons for decades : https : / / t.co / xxunk … xxmaj similar to those used on # xxunk,xxbos @simbad_reb @vegasrebs it 's worked for us for the last 18yrs has n't it ?,xxbos @igottatravel @mtats xxmaj uber is covering the cost of all rides .,xxbos @englishemma nope . i respect them for making the rides free , i can also see how they wanted to get more drivers in
y: CategoryList
Support,Support,Support,Support,Support
Path: .;

Valid: LabelList (848 items)
x: TextList
xxbos xxunk maybe tomorrow ?,xxbos xxunk @zellieimani got me all wrong the robbery suspect is wearing xxunk but michael is wearing xxunk xxunk few mins later,xxbos @nbcnews so much for xxmaj local and xxmaj xxunk xxmaj media xxunk to a xxup no xxmaj publish of the building . xxmaj brilliant idiots as usual xxup xxunk

In [15]:
data_clas.show_batch()

text,target
xxbos .@marrty96 xxup and xxup yet xxup its xxup not xxup on / xxup in xxup the xxup tweet / xxup headline . xxup as xxup usual . & & xxup dont xxup fucking xxup tell xxup me xxup to xxup relax . \n xxup this xxup is xxup the xxup same xxup network xxup that xxup reported xxup xxunk,Support
xxbos “ xxup @afp : xxmaj police stand guard as hostages are held in a cafe in xxmaj sydney . xxmaj follow xxup afp 's xxup live xxup report : http : / / t.co / xxunk http : / / t.co / xxunk ”,Support
"xxbos xxmaj looks like every xxmaj charlie xxmaj hebdo cartoonist has been killed : 1 . xxmaj charb - was on xxmaj al - xxmaj qaeda 's hit list ; 2 . xxmaj cabu , 3 . xxmaj wolinski , 4 . xxmaj tignous",Comment
"xxbos xxup sydney xxup hostage - xxup taker \n - xxmaj man xxmaj xxunk , xxunk \n - xxmaj originally from xxmaj iran \n - xxmaj self - styled sheikh \n - xxmaj accused of xxunk assaulting 7 women \n xxup developing ..",Comment
xxbos xxmaj this is the soldier who died today at the # xxmaj ottawa xxmaj war xxmaj memorial . xxmaj his name is xxmaj cpl . xxmaj nathan xxmaj cirillo . # xxup rip # xxunk http : / / t.co / xxunk,Comment


In [16]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.984152,0.920869,0.647406,00:06


In [17]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.887027,0.878570,0.682783,00:07


In [18]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.815512,0.866792,0.695755,00:08
1,0.842021,0.886857,0.670991,00:07
2,0.813521,0.873211,0.694575,00:07
3,0.760300,0.897621,0.694575,00:07
4,0.692841,0.902339,0.689858,00:07
5,0.598519,0.957939,0.695755,00:07
6,0.504438,0.996185,0.682783,00:08
7,0.461969,1.047890,0.681604,00:07
8,0.397825,1.015314,0.682783,00:07
9,0.444757,1.024292,0.678066,00:07


In [0]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [0]:
from sklearn.metrics import accuracy_score, f1_score

In [21]:
accuracy_score(test['label'], l)

0.7117437722419929

In [22]:
f1_score(test['label'], l, average='macro')

0.5199528493008301